# Латентно-семантический анализ (Latent Semantic Analysis или LSA)

В этом тюториале мы:
- познакомимся со стандартным датасетом _20 Newsgroups_
- научимся использовать метод латентно-семантическго анализа (LSA) для представления текстов в виде плотных (dense) эмбеддингов, компонентами которых являются скрытые признаки-тематики
- научимся использовать LSA-эмбеддинги для решения задач семантической близости и текстового ранжирования

При подготовке этого тюториала использовались материалы из (отличной) книги _Natural Language Processing in Action_ (авторы _Hobson Lane_ и _Maria Dyshel_): https://www.goodreads.com/book/show/59694556-natural-language-processing-in-action-second-edition

Импортируем модули которые нам понадобятся впоследствии:

In [3]:
from timeit import default_timer as timer

import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise

## Датасет 20 Newgroups

Наш тюториал будет основан на классическом датасете 20 Newsgroups.

Этот датасет был собран еще в начале 90-х годов, и представляет из себя коллекцию из примерно 20000 текстовых сообщений, отправленных в 20 разных каналах (newgroups) в сети Usenet (https://en.wikipedia.org/wiki/Usenet_newsgroup). Название канала (напр. _sci.space_, т.е. канал про космос) используется как метка класса.

Другими словами, это небольшой текстовый датасет для мультиклассовой классификации, всего 20 классов (по числу каналов).

Этот датасет приобрел очень большую популярность в мире NLP, и широко используется для иллюстрации работы различных алгоритмов, моделей и т.п.

Кроме того, этим датасетом очень удобно пользоваться, т.к. он доступен "из коробки" в библиотеке _scikit-learn_: https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset

Попробуем загрузить датасет:

In [4]:
# Filter e-mail headers, signature blocks and quotes
data_remove = ('headers', 'footers', 'quotes')

# Fetcher options
data_kwargs = { 'remove': data_remove, 'shuffle': True, 'random_state': 22 }

# Fetch train subset. 
# Bunch contains:
#   - data          -- list of text documents
#   - filenames     -- list of sample filenames
#   - target        -- numeric target in [0, 19]
#   - target_names  -- list of target names
train_bunch = datasets.fetch_20newsgroups(subset="train", **data_kwargs)
print(f"fetched 20 newsgroups dataset: num_docs = {len(train_bunch.data)}")

fetched 20 newsgroups dataset: num_docs = 11314


Обратите внимание, что:
- датасет поделен на train и test части, нас дальше будет интересовать только train
- мы скачиваем очищенный вариант датасета с удаленной из сообщений служебной информацией, такой как заголовки писем и т.п.

Всего у нас скачалось ~11 тыс. документов.

Посмотрим теперь на метки классов:

In [5]:
print(train_bunch.target)
print(train_bunch.target_names)

[10 16 11 ...  9 15  6]
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Видно, что таргеты доступны как в виде текстов (названий каналов), так и в виде числовых идентификаторов.

Посмотрим теперь на типичные тексты сообщений:

In [6]:
docs = train_bunch.data
print(docs[0])

If the Islanders beat the Devils tonight, they would finish with
identical records.  Who's the lucky team that gets to face the Penguins
in the opening round?   Also, can somebody list the rules for breaking
ties.



К какому классу относится наше сообщение?

Это легко понять следующим образом:

In [7]:
y_0 = train_bunch.target[0]
print(train_bunch.target_names[y_0])

rec.sport.hockey


Как (наверное) нетрудно было догадаться, это сообщение про хоккей, отправленное в канал _rec.sport.hockey_

## LSA-эмбеддинги

Попробуем теперь воспользоваться методом LSA и получить эмбеддинги наших текстов.

Первым делом надо превратить текст в матрицу термин-документ, т.е. получить для него разреженное признаковое представление.<br>
Сразу обратим внимание, что в качестве весов в ячейках матрицы термин-документ можно бы было использовать любой из методов взвешивания, изученных нами в лекции про модели векторого пространства, например:
- бинарные веса 1 (слово есть в документе) или 0 (слова нет в документе), которые проще всего получить с помощью класса _CountVectorizer_ из библиотеки _scikit-learn_
- TFы, т.е. частоты слова в документах
- TF-IDFы с разными способами нормализации, сглаживания и т.п.

На практике, как правило, в качестве признаков лучше всего работают TF-IDFы, поэтому мы дальше будем использовать именно их.<br>
Векторизуем наши тексты с помощью класса _TfidfVectorizer_:

In [8]:
# Prepare vectorizer
vectorizer = TfidfVectorizer(min_df=0.002, max_df=0.5, stop_words='english', decode_error='ignore')

# Vectorize
start = timer()
X_docs = vectorizer.fit_transform(docs)
print(type(X_docs))
print(f"vectorized: X_docs.shape = {X_docs.shape} elapsed = {timer() - start:.3f}")

<class 'scipy.sparse._csr.csr_matrix'>
vectorized: X_docs.shape = (11314, 5358) elapsed = 0.769


При векторизации мы сразу выкидываем стоп-слова (параметр _stop_words_), а также слишком редкие (параметр _min_df_) и слишком часто встречающиеся слова (_max_df_).

В результате у нас получилась матрица термин-документ размером 11314x5358, т.е. 11314 документов и 5358 терминов (слов).

Обратим внимание, что при желании мы могли бы использовать тут стемминг, лемматизацию или какую-то кастомную токенизацию, но не делаем этого чтобы не загромождать тюториал.

Наша матрица содержит TF-IDF весах, причем все векторы документы нормализованы с использованием L2-нормы (это дефолтное поведение векторизатора):

In [9]:
print(X_docs.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Видно, что матрица крайне разреженная, и действительно:

In [10]:
print(X_docs.nnz)

516701


Т.е. у нас заполнено только 516701 / (11314 * 5358) = 0.0085 элементов!

Посмотрим сразу и на термины, которые использует векторизатор:

In [11]:
vocab = vectorizer.get_feature_names_out()
docs_df = pd.DataFrame(X_docs.toarray(), columns=vocab)
docs_df.head()

,00,000,01,02,03,04,05,06,07,08,...,yesterday,york,young,younger,youth,yup,zero,zip,zone,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Теперь начинается самое интересное.

Применим к нашей матрице SVD-разложение:

In [12]:
# Prepare SVD
svd = TruncatedSVD(n_components=20, n_iter=100, random_state=22)

# Run SVD
start = timer()
E_docs = svd.fit_transform(X_docs)
print(E_docs.round(3))
print(f"decomposed: E_docs.shape = {E_docs.shape} elapsed = {timer() - start:.3f}")

[[ 0.043  0.015 -0.051 ... -0.002  0.005  0.013]
 [ 0.015  0.008 -0.    ...  0.012 -0.001  0.002]
 [ 0.098 -0.011 -0.029 ... -0.005 -0.035 -0.045]
 ...
 [ 0.084  0.002  0.025 ...  0.005  0.011  0.1  ]
 [ 0.159  0.157  0.18  ... -0.021 -0.013  0.004]
 [ 0.     0.     0.    ...  0.     0.     0.   ]]
decomposed: E_docs.shape = (11314, 20) elapsed = 4.253


Мы используем класс _TruncatedSVD_ из библиотеки _scikit-learn_, а в качестве параметра _n_components_ передаем ему желаемый размер нашего скрытого пространства.

Размер скрытого пространства является, по сути, гиперпараметром нашего метода и его можно попробовать подобрать под целевую задачу, в которой мы впоследствии будем использовать наши эмбеддинги. На практике, как правило, хорошо работают эмбеддинги размером 100-300 элементов.

В данном примере мы будем использовать _n_components_ равное 20, потому что:
- нам заранее известно, что у нас всего 20 классов, т.е. кажется что 20 -- это минимальное количество тематик, которыми можно описать нашу коллекцию
- с другой стороны, хотим показать, что даже такое радикальное уменьшение размерности (с 5358 до 20!) позволит не только использовать получившиеся эмбеддинги в задаче ранжирования, но и наделит их "суперспособностю" к определению степени семантической близости, которой не обладали исходные sparse-эмбеддинги. 

Посмотрим на получившиеся в результате SVD-разложения сингулярные значения:

In [13]:
print(svd.singular_values_)

[13.21846562  7.81291588  6.39003008  6.30542922  6.02990338  5.84798858
  5.59991578  5.54587373  5.36759762  5.15852419  5.07190324  4.82720005
  4.75774718  4.66189747  4.60366438  4.57800603  4.51058406  4.47745504
  4.36563922  4.36451438]


Мы видим, что они убывают, как и должно быть, причем самый главный компонент выглядит особенно "мощным".

Теперь попробуем получить матрицу термин-тематика (а не термин-документ как раньше). Для этого мы воспользуемся матрицей _Vt_ нашего разложения _X = USVt_.

Обратите внимание, что _TfidfVectorizer_ выдал нам _транспонированную_ матрицу термин-документ, поэтому, в отличие от того варианта разложения которое мы разбирали на лекции, тут у нас "все наоборот", т.е. матрица _U_ соответствует документам, а матрица _Vt_ терминам.

In [14]:
Vt = svd.components_.T
term2topic = pd.DataFrame(data=Vt, index=vocab, columns = [f'topic_{r}' for r in range(0, Vt.shape[1])])
term2topic.tail()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
yup,0.003378,0.002858,-0.002525,-0.002105,-0.001963,-0.000207,-0.005215,0.000469,0.000207,0.000653,-0.001189,-0.002635,-0.002220,-0.003464,0.001621,0.001621,0.002894,0.000645,-0.002918,0.007704
zero,0.005956,-0.000196,-0.003918,-0.003631,0.001438,0.001204,-0.000401,-0.004462,-0.000463,0.005168,-0.002806,0.000237,0.011480,-0.001789,0.004662,-0.002423,-0.002463,-0.000432,-0.003558,0.000163
zip,0.009843,-0.025317,0.010224,0.014123,0.011043,-0.018178,-0.007631,-0.036915,-0.011516,-0.007796,-0.020277,-0.021518,-0.037796,0.040020,0.017798,0.061394,-0.006429,0.012478,0.051588,-0.051003
zone,0.006381,0.003135,-0.007423,-0.005353,-0.001497,-0.003252,0.000903,-0.003632,-0.003073,-0.013344,0.003078,0.005641,0.010431,-0.004679,-0.008713,0.008451,0.002264,0.012437,-0.005999,-0.008993
zoom,0.002315,-0.003965,-0.001239,-0.000660,-0.002567,0.000906,0.006911,-0.000178,-0.003172,0.004746,0.003572,-0.002368,-0.000511,0.000170,-0.003110,-0.005779,0.001702,0.003213,0.003137,-0.002417


Мы видим, насколько каждый термин соответсвует каждой из 20 тематик.

Возьмем теперь 0-ю (самую сильную, т.е. с самым большим сингулярным значением) тематику и посмотрим, какие термины наиболее тестно с ней связаны, т.е. по сути получим облако слов, лучше всего описывающих данную тематику:

In [15]:
concept_series = term2topic[f'topic_0']
concept_series = concept_series.sort_values(ascending=False)
concept_series[:10]

don       0.163821
like      0.161092
just      0.160932
know      0.159600
people    0.151956
think     0.136384
does      0.127629
use       0.114281
good      0.113563
time      0.109619
Name: topic_0, dtype: float64

Признаемся честно, тут ничего не понятно :-)

Однако не будем расстраиваться раньше времени и посмотрим на следующую по силе тематику:

In [16]:
concept_series = term2topic[f'topic_1']
concept_series = concept_series.sort_values(ascending=False)
concept_series[:10]

god        0.253677
people     0.188681
jesus      0.117584
think      0.107536
say        0.090334
believe    0.085182
don        0.084835
bible      0.072719
did        0.070173
said       0.070052
Name: topic_1, dtype: float64

Видим, что у нас отчетливо выделилась тематика, связанная с религией, что неудивительно с учетом того, что в датасете были использованы сообщения из каналов _alt.atheism_, _soc.religion.christian_ и _talk.religion.misc_!

Посмотрим еще на какую-нибудь тематику:

In [17]:
concept_series = term2topic[f'topic_5']
concept_series = concept_series.sort_values(ascending=False)
concept_series[:10]

drive         0.436100
scsi          0.240459
drives        0.134428
disk          0.133809
key           0.128115
hard          0.127249
ide           0.124890
controller    0.119298
chip          0.116587
card          0.108599
Name: topic_5, dtype: float64

Тут у нас, очевидно, что-то околокомпьютерное.

Таким образом, мы видим, что метод LSA действительно позволяет выделять осмысленные тематики, несмотря на то, что мы использовали эмбеддинги размером всего 20 компонент!

## Ранжирование с помощью LSA

Теперь перейдем к самому интересному: попробуем ранжировать наши документы, используя в качестве ранков косинусное расстояние между LSA-векторами запросов и документов, и сравним с тем что получилось бы с использованием обычных TF-IDF-векторов.

Допустим, у нас есть текстовый запрос query.

Напишем две фукнции _search_sparse(query)_ и _search_dense(query)_, которые находят топ-К ближайших к запросу документов с использованием, соответственно, TF-IDF и LSA векторов.

Начнем с _search_sparse(query)_, она:
- векторизует наш запрос с помощью обученного ранее векторизатора
- считает попарную близость между TF-IDF-вектором запроса и TF-IDF-векторами документов
- выводит на экран запрос и ранжированный список документов

In [18]:
def search_sparse(query):
        # Vectorize query
        X_query = vectorizer.transform([query])
        print(f"vectorized query: X_query.shape = {X_query.shape}")

        # Query-docs similarity
        S = pairwise.cosine_similarity(X_query, X_docs)
        print(f"got similarities: S.shape = {S.shape}")

        # Rank docs
        scores = S[0]
        indexes = np.argsort(scores)[::-1]
        ranked_docs = np.array(docs)[indexes]
        ranked_doc_scores = scores[indexes]

        # Output query and list of ranked docs
        print(f"query = '{query}'")
        for i, doc in enumerate(ranked_docs[0:3]):
            score = ranked_doc_scores[i]
            print(f"SPARSE: [{i}]: doc = '{doc}' score = {score:.3f}")

И, аналогично, напишем функцию _search_dense(query)_, которая делает все то же самое, но уже в пространстве LSA-эмбеддингов:

In [19]:
def search_dense(query):
        # Vectorize query
        X_query = vectorizer.transform([query])
        print(f"vectorized query: X_query.shape = {X_query.shape}")

        # Embed query
        E_query = svd.transform(X_query)
        print(f"SVD-vectorized query: E_query.shape = {E_query.shape}")

        # Query-docs similarity
        S = pairwise.cosine_similarity(E_query, E_docs)
        print(f"got latent similarities: S.shape = {S.shape}")

        # Rank docs
        scores = S[0]
        indexes = np.argsort(scores)[::-1]
        ranked_docs = np.array(docs)[indexes]
        ranked_doc_scores = scores[indexes]

        # Output query and list of ranked docs
        print(f"query = '{query}'")
        for i, doc in enumerate(ranked_docs[0:3]):
            score = ranked_doc_scores[i]
            print(f"DENSE: [{i}]: doc = '{doc}' score = {score:.3f}")

Применим наши функции к запросу "mars", начем со sparse-варианта:

In [20]:
search_sparse("mars")

vectorized query: X_query.shape = (1, 5358)
got similarities: S.shape = (1, 11314)
query = 'mars'
SPARSE: [0]: doc = 'What is the deal with life on Mars?  I save the "face" and heard 
associated theories. (which sound thin to me)

Are we going back to Mars to look at this face agian?
Does anyone buy all the life theories?
' score = 0.514
SPARSE: [1]: doc = '
The "face" is an accident of light and shadow.  There are many "faces" in
landforms on Earth; none is artificial (well, excluding Mount Rushmore and
the like...).  There is also a smiley face on Mars, and a Kermit The Frog.

The question of life in a more mundane sense -- bacteria or the like -- is
not quite closed, although the odds are against it, and the most that the
more orthodox exobiologists are hoping for now is fossils.

There are currently no particular plans to do any further searches for life.


Mars Observer, currently approaching Mars, will probably try to get a better
image or two of the "face" at some point.  It's n

Видим, что в топе выдачи вполне себе релевантные документы про планету Марс, как и ожидалось.

Но что же выдаст dense-вариант?

In [21]:
search_dense("mars")

vectorized query: X_query.shape = (1, 5358)
SVD-vectorized query: E_query.shape = (1, 20)
got latent similarities: S.shape = (1, 11314)
query = 'mars'
DENSE: [0]: doc = 'Other idea for old space crafts is as navigation beacons and such..
Why not?? If you can put them on "safe" "pause" mode.. why not have them be
activated by a signal from a space craft (manned?) to act as a naviagtion
beacon, to take a directional plot on??' score = 0.982
DENSE: [1]: doc = '
If raw materials where to cost enough that getting them from space would
be cost effective then the entire world economy would colapse long
before the space mines could be built.

  Allen
' score = 0.981
DENSE: [2]: doc = ': Announce that a reward of $1 billion would go to the first corporation 
: who successfully keeps at least 1 person alive on the moon for a year. 
: Then you'd see some of the inexpensive but not popular technologies begin 
: to be developed. THere'd be a different kind of space race then!

I'm an advocate of th

Мы видим, что находятся документы про космические исследования, причем ни в одном из документов в топ-3 нет слова "mars"!

Очевидно, что это не может быть случайностью, а это значит что мы действительно смогли сматчить запросы и документы не по факту наличия ключевых слов, а по их семантической близости!

Попробуем еще один запрос:

In [22]:
search_sparse("penguins")

vectorized query: X_query.shape = (1, 5358)
got similarities: S.shape = (1, 11314)
query = 'penguins'
SPARSE: [0]: doc = 'How do you beat the Penguins?


Crash the team plane.
' score = 0.507
SPARSE: [1]: doc = 'The subject line says it all.  Is it terribly difficult to get tickets
to Penguins games, especially now that they are in the playoffs?  Would
it be easy to find scalpers outside of the Igloo selling tickets?' score = 0.315
SPARSE: [2]: doc = '
Why?  I'm calling this Penguins ... in 6.  Only that with the way 
things stand, the only radio game at that hour is from the Devils
on WABC, 770 AM.  It'd be nice to have a Sony Watchman, but ...

No need to be paranoid, Robbie.  Don't judge me by my geographic
coordinates ...

Jets over Nordiques in the final ... 7.

gld' score = 0.296


В топе документы про хоккей, т.к. _Pittsburgh Penguins_ -- это одна из комманд NHL.

Повторим поиск с использованием LSA-эмбеддингов:

In [23]:
search_dense("penguins")

vectorized query: X_query.shape = (1, 5358)
SVD-vectorized query: E_query.shape = (1, 20)
got latent similarities: S.shape = (1, 11314)
query = 'penguins'
DENSE: [0]: doc = '.
.
.

ESPN had the Houston Astros @ Chicago Cubs game scheduled for last night on the
west coast. 

Since the game was rained out, they showed the Toronto Maple Leafs at the
Detroit Red Wings game instead.' score = 0.992
DENSE: [1]: doc = '
Not clear to me at all.  I'd certainly rather have a team who was winning
4-1 games than 2-1 games.  In the 2-1 game, luck is going to play a much
bigger role than in the 4-1 game. ' score = 0.987
DENSE: [2]: doc = 'First game, first at bat.' score = 0.983


На 1-м месте у нас тоже документ про хоккей (т.к. _Toronto Maple Leafs_ и _Detroit Red Wings_ -- это тоже команды NHL), на 2-м просто что-то похожее на хоккей, а вот 3-й документ уже кажется про бейсбол (судя по слову _bat_ т.е. "бита"), но это тоже спорт, т.е. и тут мы смогли распознать какие-то оттенки семантики.